# Imports

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

%reload_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

from typing import List, Callable

import sys
import os

#https://github.com/FAU-DLM/wsi_processing_pipeline
sys.path.append("../")
sys.path.append("../wsi_processing_pipeline/")
sys.path.append("../wsi_processing_pipeline/tile_extraction")
import wsi_processing_pipeline
import tile_extraction
import preprocessing
from wsi_processing_pipeline.tile_extraction import tiles, util, slide, filter
from wsi_processing_pipeline.tile_extraction.tiles import WsiInfo
from wsi_processing_pipeline.preprocessing import *
import wsi_processing_pipeline.preprocessing.files_getter
from wsi_processing_pipeline. preprocessing.files_getter import *
import wsi_processing_pipeline.preprocessing.name_getter
from wsi_processing_pipeline.preprocessing.name_getter import NameGetter
from wsi_processing_pipeline.preprocessing.objects import ObjectManager
from wsi_processing_pipeline.preprocessing.tile_image_block import TileImage, label_tl_image, tile_image, TileTransform
from wsi_processing_pipeline.preprocessing.tile_image_block import show_batch, TileImageBlock
sys.path.append('../models-pytorch/pretrained-models.pytorch')
import pretrainedmodels
from pretrainedmodels import *


import fastai2
from fastai2.vision.all import *
from typing import Dict
import pandas as pd
import numpy as np
import os
import torch
torch.backends.cudnn.benchmark=True
import torchvision
from torchvision.models import *
from torchsummary import summary
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
import matplotlib.image as mpimg
import shutil
from PIL import Image
import sklearn
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

import pathlib
from pathlib import Path
Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

from fastai.vision.image import open_image
from concurrent.futures import as_completed, ProcessPoolExecutor
from tqdm import tqdm

#against DecompressionBombWarning
Image.MAX_IMAGE_PIXELS = 10000000000   

PATH = Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/')
LABELS = PATH/'labels.xlsx'


seed = 42
np.random.seed(seed)

# Data

## organize dataset

In [3]:
labels_df = pd.read_excel(LABELS)

In [4]:
labels_df.head()

,patient_id,case_id,slide_id,methylation_status,Unnamed: 4
0,Schwemmer-Andreas-1974-03-14,N296-08,N296-08.I.1,0.0,NaN
1,Schwemmer-Andreas-1974-03-14,N296-08,N296-08.II,0.0,NaN
2,Ballmann-Artur-1934-06-04,N320-08,N320-08.4,0.0,NaN
3,Ballmann-Artur-1934-06-04,N320-08,N320-08.6,0.0,NaN
4,Brandt-Ernst-1941-07-14,N608-08,N608-08,0.0,NaN


In [5]:
MIXED = PATH/'methylated'/'AAA_mixed'

In [14]:
for p in tqdm(MIXED.ls()):
    if os.path.isdir(p):
        print(p)

/opt/conda/envs/DLM_Py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [15]:
mixed_tiles = MIXED.ls()

In [18]:
n = mixed_tiles[0];n

Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/methylated/AAA_mixed/305-10.III_1515.png')

In [22]:
n.stem.split('_')[0]

'305-10.III'

In [23]:
slide_names = set([p.stem.split('_')[0] for p in mixed_tiles])

In [24]:
slide_names

{'1028-09',
 '11-08.4',
 '1105-08.3',
 '1122-08.3',
 '1130-09.2',
 '1147-11',
 '1185-08',
 '1436-08',
 '1752-11',
 '258-10',
 '305-10.III',
 '375-09',
 '435-09',
 '532-12',
 '7-08.II.1',
 '749-09.II.3',
 '822-09',
 '842-08'}

In [25]:
len(slide_names)

18

In [27]:
sn_to_tile_paths = {}
for sn in slide_names:
    sn_to_tile_paths[sn] = []

In [28]:
for p in tqdm(MIXED.ls()):
    sn = p.stem.split('_')[0]
    sn_to_tile_paths[sn].append(p)

/opt/conda/envs/DLM_Py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [29]:
len(sn_to_tile_paths)

18

In [38]:
for k, v in sn_to_tile_paths.items():
    print(f'{k}: {len(v)}')

11-08.4: 3953
1105-08.3: 10155
1436-08: 6783
305-10.III: 4475
822-09: 12787
1185-08: 250
375-09: 16331
749-09.II.3: 12337
1752-11: 9436
532-12: 624
258-10: 1125
1130-09.2: 10966
1122-08.3: 10357
1028-09: 401
1147-11: 14262
7-08.II.1: 3596
435-09: 1181
842-08: 3905


In [30]:
MIXED

Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/methylated/AAA_mixed')

In [31]:
PATH

Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation')

In [32]:
METHYLATED = PATH/'methylated'

In [34]:
METHYLATED

Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/methylated')

In [20]:
#for sn, tps in tqdm(sn_to_tile_paths.items()):
#    os.mkdir(METHYLATED/sn)
#    for tp in tps:
#        shutil.move(str(tp), METHYLATED/sn)

In [58]:
def rename_dirs(parent_dir):
    for p in parent_dir.ls():
        new_name = p.name
        if '_Tiles' in p.name:        
            new_name = new_name.replace('_Tiles', '')
        if '_Tiles2' in p.name:
            new_name = new_name.replace('_Tiles2', '')
        if 'N' not in p.name:
            new_name = 'N' + new_name
            
        os.rename(p, p.parent/new_name)

In [55]:
rename_dirs(METHYLATED)

In [56]:
NON_METHYLATED = PATH/'non_methylated'

In [57]:
rename_dirs(NON_METHYLATED)

In [59]:
PATH

Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation')

In [4]:
TILES_1819_METHYL = PATH/'data2018_19'/'methylated'
TILES_1819_NON_METHYL = PATH/'data2018_19'/'non_methylated'

In [5]:
#rename_dirs(TILES_1819_METHYL)
#rename_dirs(TILES_1819_NON_METHYL)

In [6]:
PATH_1819 = PATH/'data2018_19'

In [7]:
tile_paths_1819 = FilesGetter().get_dirs_and_files(path=PATH_1819, 
                                                        get_files=True, 
                                                        suffix='.png', 
                                                        recursive=True)

105it [00:31,  3.37it/s]


In [8]:
len(tile_paths_1819)

515553

In [10]:
def check_image(path):
    try:
        open_image(path)
    except Exception as e:
        return path
        
    return None

In [13]:
with ProcessPoolExecutor(max_workers=32) as pool:
    with tqdm(total=len(tile_paths_1819)) as progress:
        futures = []

        for p in tile_paths_1819:
            future = pool.submit(check_image, p)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        results = []
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                results.append(result)

  3%|▎         | 15065/515553 [03:04<1:41:53, 81.87it/s] 


KeyboardInterrupt: 

In [ ]:
failed = []
for p in tqdm(tile_paths_1819):
    r = check_image(p)
    if r is not None:
        failed.append(r)
        

  0%|          | 54/515553 [00:23<48:50:05,  2.93it/s]

## tile paths

In [5]:
tile_paths=wsi_processing_pipeline.preprocessing.files_getter.FilesGetter().get_dirs_and_files(path=PATH, 
                                                                        get_files=True, 
                                                                        suffix='.png', 
                                                                        recursive=True)

202it [02:04,  1.62it/s]


In [6]:
len(tile_paths)

1938980

In [9]:
tile_paths.items[:10]

[Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/305-10.III_4312.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/435-09_1573.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/11-08.4_3483.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/305-10.III_4713.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/11-08.4_4161.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/435-09_1456.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/305-10.III_4123.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/11-08.4_325.png'),
 Path('/home/Deep_Learner/private/datasets/glioblastoma_methylation/realdata/test/MeGBM/435-09_36.png'),
 Path('/home/Deep_Learner